# **Data Collection**

## Objectives

* Import packages
* Set working directory
* Fetch data from Kaggle and save as raw data
* Clean data
* Split data

## Inputs

* Kaggle JSON file - authentication token

## Outputs

* Generate Dataset: inputs/cherryleaves_dataset/cherry-leaves

## Additional Comments

* The steps are needed to fetch the data, clean and divide it for ML. The next step will be Data Visualization.



---

## Import Packages ##

In [ ]:
%pip install -r /workspaces/cherryleafmildewdetector-ci-project5/requirements.txt

In [1]:
import numpy
import os

## Change working directory

We need to change the working directory from its current folder to its parent folder.

In [2]:
current_dir = os.getcwd()
current_dir

'/workspaces/cherryleafmildewdetector-ci-project5/jupyter_notebooks'

We want to make the parent of the current directory the new current directory

In [4]:
os.chdir('/workspaces/cherryleafmildewdetector-ci-project5')
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [5]:
current_dir = os.getcwd()
current_dir

'/workspaces/cherryleafmildewdetector-ci-project5'

---

# Install Kaggle

In [6]:
%pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [kaggle]2m3/4 [kaggle]
Note: you may need to restart the kernel to use updated packages.


* Download kaggle.json file from Kaggle.
* Move kaggle.json file into project.

Run below cell to change the Kaggle configuration directory to the current working directory and set permissions for the Kaggle authentication JSON.

In [8]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

* Get the dataset path from the [Kaggle Dataset URL](https://www.kaggle.com/datasets/codeinstitute/cherry-leaves).
* Set your destination folder.

Set the kaggle dataset and download it

In [9]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherryleaves_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

Dataset URL: https://www.kaggle.com/datasets/codeinstitute/cherry-leaves
License(s): unknown
  0%|                                               | 0.00/55.0M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:00<00:00, 1.84GB/s]


Unzip the dowloaded file and delete the zip file

In [10]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

---

# Data Preparation

## Data Cleaning

Check and remove all non-image files

In [11]:
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        # print(files)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location)  # remove non image file
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file", len(j))
        print(f"Folder: {folder} - has non-image file", len(i))

In [12]:
remove_non_image_file(my_data_dir='inputs/cherryleaves_dataset/cherry-leaves')

Folder: powdery_mildew - has image file 2104
Folder: powdery_mildew - has non-image file 0
Folder: healthy - has image file 2104
Folder: healthy - has non-image file 0


## Split train validation test set

In [13]:
import os
import shutil
import random
import joblib


def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):

    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print("train_set_ratio + validation_set_ratio + test_set_ratio should sum to 1.0")
        return

    # gets classes labels
    labels = os.listdir(my_data_dir)  # it should get only the folder name
    if 'test' in labels:
        pass
    else:
        # create train, test folders with classes labels sub-folder
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=my_data_dir + '/' + folder + '/' + label)

        for label in labels:

            files = os.listdir(my_data_dir + '/' + label)
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)

            count = 1
            for file_name in files:
                if count <= train_set_files_qty:
                    # move a given file to the train set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/train/' + label + '/' + file_name)

                elif count <= (train_set_files_qty + validation_set_files_qty):
                    # move a given file to the validation set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/validation/' + label + '/' + file_name)

                else:
                    # move given file to test set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/test/' + label + '/' + file_name)

                count += 1

            os.rmdir(my_data_dir + '/' + label)

The dataset will be divided up into:

* Training set: 70% of data.
* Validation set: 10% of data.
* Test set: 20% of data.

In [14]:
split_train_validation_test_images(my_data_dir=f"inputs/cherryleaves_dataset/cherry-leaves",
                                   train_set_ratio=0.7,
                                   validation_set_ratio=0.1,
                                   test_set_ratio=0.2
                                   )

---

# Conclusions

In this notebook, the following was achieved:

* Dataset successfully imported via the Kaggle API
* Dataset inspected, and no missing values were found
* Dataset split into training, validation, and test sets

### Next Steps
In the next notebook, data analysis will be carried out (Business requirement 1).